In [8]:
#importing libraries
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


In [ ]:
'''
# Base path and output folder
base_dir = 'Raw Data'
combined_folder = 'combined_data'
os.makedirs(combined_folder, exist_ok=True)

# Step 1: Fix missing 'month' column for 2021 and 2022
def fix_missing_month_column(base_dir, years_to_fix):
    for year in years_to_fix:
        print(f"\n Fixing missing 'month' column for {year}...")
        year_path = os.path.join(base_dir, year)

        for root, _, files in os.walk(year_path):
            for file in files:
                if file.endswith('.csv') and 'ytd' not in file.lower():
                    file_path = os.path.join(root, file)
                    try:
                        df = pd.read_csv(file_path)
                        df.columns = df.columns.str.lower().str.strip()

                        if 'month' not in df.columns:
                            folder_name = os.path.basename(root)
                            df['month'] = folder_name
                            df.to_csv(file_path, index=False)
                            print(f"✓ Added 'month' to {file} (Folder: '{folder_name}')")
                        else:
                            print(f"• Skipped (already has 'month'): {file}")
                    except Exception as e:
                        print(f"✗ Error processing {file}: {e}")

# Step 2: Combine files with column consistency
def combine_yearly_files(base_dir, years_to_combine, combined_folder):
    for year in years_to_combine:
        dataframes = []
        year_path = os.path.join(base_dir, year)
        expected_columns = None
        mismatched_files = []

        print(f"\n Combining files for {year}...")

        for root, _, files in os.walk(year_path):
            for file in files:
                if file.endswith('.csv') and 'ytd' not in file.lower():
                    file_path = os.path.join(root, file)
                    try:
                        df = pd.read_csv(file_path)
                        df.columns = df.columns.str.lower().str.strip()

                        # Set expected columns from the first file
                        if expected_columns is None:
                            expected_columns = list(df.columns)
                        else:
                            # Check if columns match
                            if list(df.columns) != expected_columns:
                                mismatched_files.append((file_path, list(df.columns)))
                                continue  # Skip this file

                        dataframes.append(df)
                    except Exception as e:
                        print(f"✗ Error reading {file_path}: {e}")

        # Report any mismatched files
        if mismatched_files:
            print(f" Column mismatches in {year} (skipping these files):")
            for path, cols in mismatched_files:
                print(f"  - {path}\n    Columns: {cols}")

        # Combine consistent files
        if dataframes:
            combined_df = pd.concat(dataframes, ignore_index=True)
            output_path = os.path.join(combined_folder, f"{year}_combined.csv")
            combined_df.to_csv(output_path, index=False)
            print(f" {year} combined and saved to {output_path}")
        else:
            print(f" No consistent non-YTD files found for {year}")

# Run steps
fix_missing_month_column(base_dir, years_to_fix=['2021', '2022'])
combine_yearly_files(base_dir, years_to_combine=['2020', '2021', '2022', '2023', '2024'], combined_folder=combined_folder)
''' 


 Fixing missing 'month' column for 2021...
• Skipped (already has 'month'): dot1_0421.csv
• Skipped (already has 'month'): dot2_0421.csv
• Skipped (already has 'month'): dot3_0421.csv
• Skipped (already has 'month'): dot1_0821.csv
• Skipped (already has 'month'): dot2_0821.csv
• Skipped (already has 'month'): dot3_0821.csv
• Skipped (already has 'month'): dot1_1221.csv
✓ Added 'month' to dot1_2021.csv (Folder: 'Dec 2021')
• Skipped (already has 'month'): dot2_1221.csv
• Skipped (already has 'month'): dot2_2021.csv
• Skipped (already has 'month'): dot3_1221.csv
• Skipped (already has 'month'): dot3_2021.csv
• Skipped (already has 'month'): dot1_0221.csv
• Skipped (already has 'month'): dot2_0221.csv
• Skipped (already has 'month'): dot3_0221.csv
• Skipped (already has 'month'): dot1_0121.csv
• Skipped (already has 'month'): dot2_0121.csv
• Skipped (already has 'month'): dot3_0121.csv
• Skipped (already has 'month'): dot1_0721.csv
• Skipped (already has 'month'): dot2_0721.csv
• Skipped

#### Reading Datasets

In [12]:
shipping_20 = pd.read_csv('combined_data/2020_combined.csv')
shipping_21 = pd.read_csv('combined_data/2021_combined.csv')
shipping_22 = pd.read_csv('combined_data/2022_combined.csv')
shipping_23 = pd.read_csv('combined_data/2023_combined.csv')
shipping_24 = pd.read_csv('combined_data/2024_combined.csv')

### Data Cleaning

In [13]:

# Dictionary of your loaded DataFrames
datasets = {
    '2020_combined.csv': shipping_20,
    '2021_combined.csv': shipping_21,
    '2022_combined.csv': shipping_22,
    '2023_combined.csv': shipping_23,
    '2024_combined.csv': shipping_24
}

# Loop through each DataFrame
for name, df in datasets.items():
    total_rows = len(df)

    # Check for missing values
    missing_count = df.isnull().sum()
    missing_percent = (missing_count / total_rows) * 100
    total_missing = missing_count.sum()

    # Check for duplicates
    duplicate_rows = df.duplicated().sum()

    print(f"\n {name}")
    print(f"→ Total rows: {total_rows}")
    print(f"→ Total missing values: {total_missing}")

    if total_missing > 0:
        missing_df = pd.DataFrame({
            'Missing Values': missing_count,
            'Percentage (%)': missing_percent
        })
        missing_df = missing_df[missing_df['Missing Values'] > 0]
        print("Missing values by column:")
        print(missing_df.sort_values(by='Percentage (%)', ascending=False))
    else:
        print("No missing values.")
    
    print(f"→ Duplicate rows: {duplicate_rows}")



 2020_combined.csv
→ Total rows: 180588
→ Total missing values: 263171
Missing values by column:
          Missing Values  Percentage (%)
mexstate          128851       71.350810
df                 68901       38.153698
canprov            65419       36.225552
→ Duplicate rows: 0

 2021_combined.csv
→ Total rows: 300092
→ Total missing values: 436906
Missing values by column:
          Missing Values  Percentage (%)
mexstate          212481       70.805286
df                114032       37.999014
canprov           110393       36.786386
→ Duplicate rows: 0

 2022_combined.csv
→ Total rows: 338101
→ Total missing values: 491412
Missing values by column:
          Missing Values  Percentage (%)
mexstate          241444       71.411797
df                127609       37.742864
canprov           122359       36.190073
→ Duplicate rows: 0

 2023_combined.csv
→ Total rows: 341668
→ Total missing values: 495541
Missing values by column:
          Missing Values  Percentage (%)
mexstate       

#### BUSINESS QUESTIONS
###### 1.	Which trade type brings in the highest revenue.(Trade Deficits)
###### 2.	What are the top 10 most frequently traded commodities.(Supply Chain Challenges)
###### 3.	The relationship between domestic and foreign goods to revenue. (Cost Efficiency)
###### 4.	The relationship between domestic and foreign goods to freight charges. (Pricing Strategy)
###### 5.	Containerized and non- containerized shipment vrs freight charges. (Infrastructure Investment)
###### 6.	Which countries imports or exports more.(Market Prioritization)
###### 7.	Countries vrs df(Policy and Compliance)
###### 8.	What are the peak months for both exports and imports.
###### 9.	Frequently used mode of transport
###### 10.	


##### Handling missing values
Dropping the Mexstate and Canprove columns as they do not help answer our business questions.
Keeping the DF column and treating the missing values as merchandise with unknown origin

In [14]:

# Columns to drop
columns_to_drop = ['mexstate', 'canprov']

# Dictionary of DataFrames
datasets = {
    '2020_combined.csv': shipping_20,
    '2021_combined.csv': shipping_21,
    '2022_combined.csv': shipping_22,
    '2023_combined.csv': shipping_23,
    '2024_combined.csv': shipping_24
}

# Process each in-memory DataFrame
for name, df in datasets.items():
    # Drop only the columns that exist
    existing_cols = [col for col in columns_to_drop if col in df.columns]
    df.drop(columns=existing_cols, inplace=True)
    print(f" Sucessfully Dropped {existing_cols} from {name}")


 Sucessfully Dropped ['mexstate', 'canprov'] from 2020_combined.csv
 Sucessfully Dropped ['mexstate', 'canprov'] from 2021_combined.csv
 Sucessfully Dropped ['mexstate', 'canprov'] from 2022_combined.csv
 Sucessfully Dropped ['mexstate', 'canprov'] from 2023_combined.csv
 Sucessfully Dropped ['mexstate', 'canprov'] from 2024_combined.csv


In [15]:
# Replacement value
replacement_value = 'Unspecified'

# Dictionary of DataFrames
datasets = {
    '2020_combined.csv': shipping_20,
    '2021_combined.csv': shipping_21,
    '2022_combined.csv': shipping_22,
    '2023_combined.csv': shipping_23,
    '2024_combined.csv': shipping_24
}

# Replace missing 'df' values
for name, df in datasets.items():
    if 'df' in df.columns:
        missing_count = df['df'].isna().sum()
        df['df'].fillna(replacement_value, inplace=True)
        print(f" Replaced {missing_count} missing 'df' values in {name}")
    else:
        print(f" 'df' column not found in {name}")


 Replaced 68901 missing 'df' values in 2020_combined.csv
 Replaced 114032 missing 'df' values in 2021_combined.csv
 Replaced 127609 missing 'df' values in 2022_combined.csv
 Replaced 128111 missing 'df' values in 2023_combined.csv
 Replaced 96452 missing 'df' values in 2024_combined.csv


##### DATA MAPPING
##### Defining data variables
creating a temporary mapping to avoid changes to the dataset

In [46]:
# --- 1. Define all mappings ---
'''
commodity_map = {
    1: 'Live animals', 2: 'Meat', 3: 'Fish', 4: 'Dairy, Honey, etc.',
    5: 'Other animal products', 6: 'Live plants', 7: 'Vegetables',
    8: 'Fruits and nuts', 9: 'Coffee, tea, spices', 10: 'Cereals',
    11: 'Milling products', 12: 'Oil seeds', 13: 'Gums and resins',
    14: 'Veg. plaiting', 15: 'Fats and oils', 16: 'Prepared meat/fish',
    17: 'Sugar', 18: 'Cocoa', 19: 'Cereal preparations', 20: 'Veg./fruit preparations',
    21: 'Misc. edible', 22: 'Beverages', 23: 'Food waste/feed', 24: 'Tobacco',
    25: 'Salt/stone/cement', 26: 'Ores', 27: 'Mineral fuels', 28: 'Inorganic chemicals',
    29: 'Organic chemicals', 30: 'Pharma products', 31: 'Fertilizers',
    32: 'Dyes & paints', 33: 'Essential oils & cosmetics', 34: 'Soaps & waxes',
    35: 'Starches, glues, enzymes', 36: 'Explosives & matches', 37: 'Photographic goods',
    38: 'Misc chemicals', 39: 'Plastics', 40: 'Rubber', 41: 'Raw hides & leather',
    42: 'Leather articles', 43: 'Furskins', 44: 'Wood', 45: 'Cork', 46: 'Basketware',
    47: 'Pulp & scrap paper', 48: 'Paper articles', 49: 'Books & printed goods',
    50: 'Silk', 51: 'Wool', 52: 'Cotton', 53: 'Other textiles', 54: 'Man-made filaments',
    55: 'Man-made fibers', 56: 'Ropes & twine', 57: 'Carpets', 58: 'Special woven fabrics',
    59: 'Industrial textiles', 60: 'Knitted fabrics', 61: 'Knit apparel', 62: 'Woven apparel',
    63: 'Other textile articles', 64: 'Footwear', 65: 'Headgear', 66: 'Umbrellas & sticks',
    67: 'Feathers & artificial flowers', 68: 'Stone & cement articles', 69: 'Ceramics',
    70: 'Glassware', 71: 'Jewelry & coins', 72: 'Iron & steel', 73: 'Iron/steel articles',
    74: 'Copper', 75: 'Nickel', 76: 'Aluminum', 77: 'Reserved', 78: 'Lead',
    79: 'Zinc', 80: 'Tin', 81: 'Other base metals', 82: 'Tools & cutlery',
    83: 'Misc base metal', 84: 'Machinery', 85: 'Electrical machinery', 86: 'Railway equipment',
    87: 'Vehicles', 88: 'Aircraft', 89: 'Ships', 90: 'Instruments', 91: 'Watches',
    92: 'Musical instruments', 93: 'Arms & ammo', 94: 'Furniture & prefab', 95: 'Toys & sports',
    96: 'Misc manufactured', 97: 'Art & antiques', 98: 'Special classification', 99: 'Temporary (imports only)'
}
'''
trade_type_mapping = {
    1: "Export",
    2: "Import"
}

disagmot_mapping = {
    1: "Vessel",
    3: "Air",
    4: "Mail (U.S. Postal Service)",
    5: "Truck",
    6: "Rail",
    7: "Pipeline",
    8: "Other",
    9: "Foreign Trade Zones (FTZs)"
}

country_mapping = {
    1220: 'Canada',
    2010: 'Mexico'
}

df_mapping = {
    1: 'domestically produced merchandise',
    2: 'foreign produced merchandise'
}

contcode_mapping = {
    'X':'Containerized',
    0: 'Non-Containerized'
}

month_mapping = {
    1: 'January', 2: 'February', 3: 'March', 4: 'April',
    5: 'May', 6: 'June', 7: 'July', 8: 'August',
    9: 'September', 10: 'October', 11: 'November', 12: 'December'
}

def get_mapped_copy(df):
    return df.copy().assign(
        trdtype     = df['trdtype'].map(trade_type_mapping),
        disagmot    = df['disagmot'].map(disagmot_mapping),
        country     = df['country'].map(country_mapping),
        df          = df['df'].map(df_mapping),
        contcode    = df['contcode'].map(contcode_mapping),
        month       = df['month'].map(month_mapping)
        # Add commodity2 if needed:
        # commodity2 = df['commodity2'].map(commodity_map)
    )



##### EXPLORATORY DATA ANALYSIS
combining existing dataframes into one

In [28]:
# Combine all years into one DataFrame
combined_shipping = pd.concat(
    [shipping_20, shipping_21, shipping_22, shipping_23, shipping_24],
    ignore_index=True
)



In [47]:
# check trade types in 2023
mapped_combined = get_mapped_copy(combined_shipping)


In [51]:
mapped_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1421552 entries, 0 to 1421551
Data columns (total 12 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   trdtype          1421552 non-null  object
 1   usastate         1421552 non-null  object
 2   depe             1421552 non-null  object
 3   disagmot         1421552 non-null  object
 4   country          1421552 non-null  object
 5   value            1421552 non-null  int64 
 6   shipwt           1421552 non-null  int64 
 7   freight_charges  1421552 non-null  int64 
 8   df               886447 non-null   object
 9   contcode         526131 non-null   object
 10  month            1421552 non-null  object
 11  year             1421552 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 130.1+ MB


In [49]:
mapped_combined.describe()

,value,shipwt,freight_charges,year
count,1.421552e+06,1.421552e+06,1.421552e+06,1.421552e+06
mean,4.334928e+06,1.746642e+06,5.651869e+04,2.022143e+03
std,4.731744e+07,4.807430e+07,1.192638e+06,1.293824e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,2.020000e+03
25%,1.994600e+04,0.000000e+00,0.000000e+00,2.021000e+03
50%,1.039605e+05,3.000000e+00,3.820000e+02,2.022000e+03
75%,6.711342e+05,9.480250e+03,3.785000e+03,2.023000e+03
max,4.951806e+09,8.563936e+09,2.275531e+08,2.024000e+03


Key Takeaways from the data description


- **Exports dominate** the dataset based on `trdtype`.
- **Truck transport** is the most frequent (`disagmot` = 5).
- **Canada** has a higher number of records compared to Mexico.
- **Trade value, weight, and freight charges** show strong **positive skewness** — driven by a small number of high-magnitude shipments.
- **Potential data quality issues** may exist in `shipwt` and `freight_charges` where the minimum is zero.
- Further **visual exploration** (e.g., histograms, boxplots) is recommended to confirm the shape and identify outliers.


In [ ]:
# Example: check trade types in 2023
mapped_23 = get_mapped_copy(shipping_23)
mapped_23['trdtype_name'].value_counts()

# Example: average freight by container type in 2021
mapped_21 = get_mapped_copy(shipping_21)
mapped_21.groupby('contcode_name')['freight_charge'].mean()


In [25]:
# seen that columns contcode and commodity2 have missing values. checking if all values were mapped

print("Unmapped commodity2:", df['commodity2'].isna().sum())
print("Unmapped trdtype:", df['trdtype'].isna().sum())
print("Unmapped disagmot:", df['disagmot'].isna().sum())
print("Unmapped country:", df['country'].isna().sum())
print("Unmapped df:", df['df'].isna().sum())
print("Unmapped contcode:", df['contcode'].isna().sum())
print("Unmapped month:", df['month'].isna().sum())


KeyError: 'commodity2'

In [ ]:
# If 'commodity' is a longer HS code, extract first 2 digits
df['commodity2'] = df['commodity2'].astype(str).str[:2].astype(int)


ValueError: invalid literal for int() with base 10: 'na'